## LIBRERIAS

In [1]:
import os
import pandas as pd

# Libreria para cargar datos a PostgreSQL
try:
    import psycopg2
    print("importando psycopg2")
except:
    !pip install psycopg2
    import psycopg2

importando psycopg2


## CARGA BASES

In [2]:
# Link del repositorio de datos
data_file = "https://github.com/MirandaCR/CustomerTrx/blob/main/Data/KPMG_VI_New_raw_data_update_final.xlsx?raw=true"

### Transacciones

In [3]:
trx = pd.read_excel(data_file,sheet_name='Transactions',skiprows=1)
trx.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [4]:
trx.dtypes

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object

### Clientes

In [5]:
customer = pd.read_excel(data_file,sheet_name='CustomerDemographic')
customer = customer.dropna()
customer.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_43464\3749502218.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  customer = pd.read_excel(data_file,sheet_name='CustomerDemographic')


,customer_id,name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
88,720,Darrel Canet,Male,67,1931-10-23,88.0,Recruiting Manager,Retail,Affluent Customer,N,No,6.0
89,1092,Katlin Creddon,Female,56,1935-08-22,84.0,VP Quality Control,Retail,Mass Customer,N,No,5.0
91,658,Donn Bonnell,Male,38,1944-01-24,76.0,Tax Accountant,Manufacturing,Affluent Customer,N,Yes,8.0
92,2413,Abbey Murrow,Male,27,1943-08-11,76.0,Environmental Specialist,Manufacturing,High Net Worth,N,Yes,17.0
93,1,Laraine Medendorp,Female,93,1953-10-12,66.0,Executive Secretary,Health,Mass Customer,N,Yes,11.0


In [6]:
customer.dtypes

customer_id                                     int64
name                                           object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
age                                           float64
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                        float64
dtype: object

### Direcciones

In [7]:
address = pd.read_excel(data_file,sheet_name='CustomerAddress',skiprows=1)
address = address.dropna()
address.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [8]:
address.dtypes

customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object

## CONEXION A POSTGRESQL Y TEST

In [9]:
conn = psycopg2.connect(
    host="localhost",
    database="Kaggle",
    user="postgres",
    password="root",
    port="5432"
)
print(conn)

cur = conn.cursor()

<connection object at 0x000001E1941568C0; dsn: 'user=postgres password=xxx dbname=Kaggle host=localhost port=5432', closed: 0>


In [10]:
# Revisando las tablas cargadas en el esquema
cur.execute("""
    SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'customer_trx'
""")

rows = cur.fetchall()
cur.close()
rows

[('trx',), ('customers',), ('address',)]

## CREACION Y CARGA DE BASES A POSTGRESQL

### TRX

In [11]:
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS customer_trx."trx";''')

cur.execute("""
    CREATE TABLE IF NOT EXISTS  customer_trx.trx (
        transaction_id INT,
        product_id INT,
        customer_id INT,
        transaction_date TIMESTAMP,
        online_order FLOAT,
        order_status VARCHAR(255),
        brand VARCHAR(255),
        product_line VARCHAR(255),
        product_class VARCHAR(255),
        product_size VARCHAR(255),
        list_price FLOAT,
        standard_cost FLOAT,
        product_first_sold_date FLOAT
    );
""")

conn.commit()
cur.close()

In [12]:
# validacion de carga
cur = conn.cursor()

cur.execute("""
    SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'customer_trx'
""")


rows = cur.fetchall()
cur.close()
rows

[('trx',), ('customers',), ('address',)]

In [13]:
# insertando datos a la tabla
cur = conn.cursor()

for index, row in trx.iterrows():
    cur.execute('''INSERT INTO customer_trx.trx (transaction_id, product_id, customer_id, transaction_date, online_order, 
                           order_status, brand, product_line, product_class, product_size, list_price, 
                           standard_cost, product_first_sold_date) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''', 
                (row['transaction_id'], 
                 row['product_id'],
                 row['customer_id'], 
                 row['transaction_date'], 
                 row['online_order'], 
                 row['order_status'], 
                 row['brand'], 
                 row['product_line'],
                 row['product_class'], 
                 row['product_size'], 
                 row['list_price'], 
                 row['standard_cost'], 
                 row['product_first_sold_date']))

# Commit the transaction
cur.close()
conn.commit()

In [14]:
# validacion de tabla con datos
df_trx = pd.read_sql('''SELECT * FROM customer_trx."trx"''', conn)
df_trx

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,2017-06-24,1.0,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40,37823.0
19996,19997,41,127,2017-11-09,1.0,Approved,Solex,Road,medium,medium,416.98,312.74,35560.0
19997,19998,87,2284,2017-04-14,1.0,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71,40410.0
19998,19999,6,2764,2017-07-03,0.0,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216.0


### Tabla de clientes

In [15]:
cur = conn.cursor()

# Ejecutar una consulta SQL para eliminar la tabla
cur.execute('''DROP TABLE IF EXISTS customer_trx."customers";''')

# creacion de tabla
create_table_query = """
    CREATE TABLE IF NOT EXISTS customer_trx.customers (
        customer_id INT PRIMARY KEY,
        name VARCHAR(255),
        gender VARCHAR(255),
        past_3_years_bike_related_purchases INT,
        DOB DATE,
        age FLOAT,
        job_title VARCHAR(255),
        job_industry_category VARCHAR(255),
        wealth_segment VARCHAR(255),
        deceased_indicator VARCHAR(255),
        owns_car VARCHAR(255),
        tenure FLOAT
    )
"""

cur.execute(create_table_query)
conn.commit()
cur.close()

In [16]:
# validacion de creacion
cur = conn.cursor()

cur.execute("""
    SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'customer_trx'
""")

rows = cur.fetchall()
cur.close()
rows

[('trx',), ('customers',), ('address',)]

In [17]:
# carga de datos a la tabla
cur = conn.cursor()

# Iterar a través de cada fila en el DataFrame y ejecutar una consulta SQL para insertar los datos en la tabla
for index, row in customer.iterrows():
    cur.execute("""
        INSERT INTO customer_trx.customers (customer_id, name, gender, past_3_years_bike_related_purchases, DOB, age, job_title, job_industry_category, wealth_segment, deceased_indicator, owns_car, tenure)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (row['customer_id'], 
          row['name'], 
          row['gender'], 
          row['past_3_years_bike_related_purchases'], 
          row['DOB'], 
          row['age'], 
          row['job_title'], 
          row['job_industry_category'], 
          row['wealth_segment'], 
          row['deceased_indicator'], 
          row['owns_car'], 
          row['tenure']
    ))

# Confirmar los cambios y cerrar la conexión
conn.commit()
cur.close()

In [18]:
# validacion de tabla con datos
df_customer = pd.read_sql('''SELECT * FROM customer_trx."customers"''', conn)
df_customer

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,name,gender,past_3_years_bike_related_purchases,dob,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,720,Darrel Canet,Male,67,1931-10-23,88.0,Recruiting Manager,Retail,Affluent Customer,N,No,6.0
1,1092,Katlin Creddon,Female,56,1935-08-22,84.0,VP Quality Control,Retail,Mass Customer,N,No,5.0
2,658,Donn Bonnell,Male,38,1944-01-24,76.0,Tax Accountant,Manufacturing,Affluent Customer,N,Yes,8.0
3,2413,Abbey Murrow,Male,27,1943-08-11,76.0,Environmental Specialist,Manufacturing,High Net Worth,N,Yes,17.0
4,1,Laraine Medendorp,Female,93,1953-10-12,66.0,Executive Secretary,Health,Mass Customer,N,Yes,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2859,2759,Melodee Hendrik,Female,16,2001-11-14,18.0,Operator,Health,Affluent Customer,N,Yes,1.0
2860,2808,Preston Tinkham,Male,68,2001-06-29,18.0,Actuary,Financial Services,Mass Customer,N,Yes,1.0
2861,3173,Angelo,Male,55,2001-10-10,18.0,Computer Systems Analyst I,Health,Mass Customer,N,Yes,1.0
2862,3949,Costa Sleightholm,Male,24,2001-12-19,18.0,Web Designer IV,Manufacturing,High Net Worth,N,Yes,1.0


### Direcciones

In [19]:
cur = conn.cursor()

# Ejecutar una consulta SQL para eliminar la tabla
cur.execute('''DROP TABLE IF EXISTS customer_trx."address";''')

# creacion de tabla
create_table_query = """
CREATE TABLE  customer_trx.address (
      customer_id INT,
      address VARCHAR(255),
      postcode INT,
      state VARCHAR(255),
      country VARCHAR(255),
      property_valuation INT
);
"""

cur.execute(create_table_query)
conn.commit()
cur.close()

In [20]:
# carga de datos a la tabla
cur = conn.cursor()

# Iterar a través de cada fila en el DataFrame y ejecutar una consulta SQL para insertar los datos en la tabla
for index, row in address.iterrows():
    cur.execute("""
        INSERT INTO customer_trx.address (customer_id, address, postcode, state, country, property_valuation) 
        VALUES (%s, %s, %s, %s, %s, %s);
    """, (row['customer_id'], 
          row['address'], 
          row['postcode'], 
          row['state'], 
          row['country'], 
          row['property_valuation']
    ))

# Confirmar los cambios y cerrar la conexión
conn.commit()
cur.close()

In [21]:
# validacion de creacion
cur = conn.cursor()

cur.execute("""
    SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'customer_trx'
""")

rows = cur.fetchall()
cur.close()
rows

[('trx',), ('customers',), ('address',)]

In [22]:
# validacion de tabla con datos
df_customer = pd.read_sql('''SELECT * FROM customer_trx."address"''', conn)
df_customer

C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...
3994,3999,1482 Hauk Trail,3064,VIC,Australia,3
3995,4000,57042 Village Green Point,4511,QLD,Australia,6
3996,4001,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,4002,8194 Lien Street,4032,QLD,Australia,7
